In [9]:
import numpy as np
import pandas as pd #data processing(reading csv file)

In [ ]:
#Reading the train classes that contains the image names and tags from the directory
train_classes = pd.read_csv('')
train_classes.head()

In [ ]:
#define a function to split the tags and store a set of the tags in a variable called labels.
#set is used to return the unique labels in the tags
labels = set()
def splitting_tags(tags):
    for tag in tags.split():
        labels.add(tag)

#we redefine the train_classes by creating a copy of it so as not to overwrite the existing one. 
#so a copy of the train classes is stored in the variable train_classes1, we convert labels which is a set to a list.
train_classes1 = train_classes.copy()
train_classes1['tags'].apply(splitting_tags)
labels = list(labels)
print(labels)
['blooming', 'agriculture', 'blow_down', 'clear', 'selective_logging', 'conventional_mine', 'road', 'partly_cloudy', 'water', 'cloudy', 'artisinal_mine', 'slash_burn', 'cultivation', 'bare_ground', 'habitation', 'haze', 'primary']

In [ ]:
#assert  that the length of the dataframe is the same as the shape
assert len(train_classes1['image_name'].unique()) == train_classes1.shape[0]

In [ ]:
##One hot encoding is performed on the labels in train classes
for tag in labels:
    train_classes1[tag] = train_classes1['tags'].apply(lambda x: 1 if tag in x.split() else 0)
    
## adding .jpg extension to the column image_name so as to have same name format as the image files
train_classes1['image_name'] = train_classes1['image_name'].apply(lambda x: '{}.jpg'.format(x))
train_classes1.head()

In [ ]:
#importing tensorflow libraries for training the dataset
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#defining the columns, that is the labels that were newly added to the train_classes via hot encoding.
columns = list(train_classes1.columns[2:]) #from index 2 to the end defines the columns
columns

In [ ]:
#define a function for fbeta scoring
def fbeta(y_true, y_pred, beta = 2, epsilon = 1e-4):
    
    beta_squared = beta**2
    
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.greater(tf.cast(y_pred, tf.float32), tf.constant(0.5)), tf.float32)
    
    tp = tf.reduce_sum(y_true * y_pred, axis = 1)
    fp = tf.reduce_sum(y_pred, axis = 1) - tp
    fn = tf.reduce_sum(y_true, axis = 1) - tp
    
    precision = tp/(tp+fp+epsilon)
    recall = tp/(tp+fn+epsilon)
    
    fb = (1+beta_squared)*precision*recall / (beta_squared*precision+recall+epsilon)
    return fb

In [ ]:
#define a function for accuracy for multi_label classification
def multi_label_acc(y_true, y_pred, epsilon = 1e-4):
    
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.greater(tf.cast(y_pred, tf.float32), tf.constant(0.5)), tf.float32)
    
    tp = tf.reduce_sum(y_true * y_pred, axis = 1)
    fp = tf.reduce_sum(y_pred, axis = 1) - tp
    fn = tf.reduce_sum(y_true, axis = 1) - tp
    
    y_true = tf.cast(y_true, tf.bool)
    y_pred = tf.cast(y_pred, tf.bool)
        
    tn = tf.reduce_sum(tf.cast(tf.logical_not(y_true), tf.float32) * tf.cast(tf.logical_not(y_pred), tf.float32), 
                       axis = 1)
    return (tp+tn)/(tp+tn+fp+fn+epsilon)

In [ ]:
#defining our model using a function build_model()
def build_model():
    model = Sequential()
    model.add(BatchNormalization(input_shape=(128, 128, 3)))
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(17, activation='sigmoid'))

    opt = Adam(lr=1e-4)

    model.compile(loss='binary_crossentropy',
              # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
              optimizer=opt,
              metrics=[multi_label_acc, fbeta])

    return model

In [ ]:
#modelcheckpoint is set to monitor the model using validation fbeta score and save the best only
save_best_check_point = ModelCheckpoint(filepath = 'best_model.hdf5', monitor = 'val_fbeta', mode = 'max',
                                       save_best_only = True, save_weights_only = True)
#initializing imagedatagenerator with a validation split of 0.2
train_image_gen = ImageDataGenerator(rescale = 1/255, validation_split = 0.2)

#generating train data generator which is 80% of the train dataset
#note that a generator contains both features and target of the data
train_generator = train_image_gen.flow_from_dataframe(dataframe=train_classes1,
                                                directory ="../input/planets-dataset/planet/planet/train-jpg",  
                                                x_col="image_name", y_col=columns, subset="training", 
                                                batch_size=16,seed=2021, shuffle=True, 
                                                class_mode="raw", target_size=(128,128))

#generating validation data which is expected to be 20% of the train dataset since validation split is 0.2
val_generator = train_image_gen.flow_from_dataframe(dataframe=train_classes1,
                                                directory ="../input/planets-dataset/planet/planet/train-jpg",  
                                                x_col="image_name", y_col=columns, subset="validation", 
                                                batch_size=16,seed=2021, shuffle=True, 
                                                class_mode="raw", target_size=(128,128))

In [ ]:
#setting up step size for training and validation image data
step_train_size = int(np.ceil(train_generator.samples / train_generator.batch_size))
step_val_size = int(np.ceil(val_generator.samples / val_generator.batch_size))
#initializing the model
model1 = build_model()
#this shows the summary of the model, simply put as the model architecture
model1.summary()

In [ ]:
#fitting our model using the parameters already defined 
model1.fit(x = train_generator, steps_per_epoch = step_train_size, validation_data = val_generator, 
           validation_steps = step_val_size,epochs = 25, 
           callbacks = [save_best_check_point])

In [ ]:
#initializing a second model so we can make predictions
model2 = build_model()
##adding .jpg extension to image name in the sample submission file
sample_submission = pd.read_csv('../input/planets-dataset/planet/planet/sample_submission.csv')
sample_submission1 = sample_submission.copy()
sample_submission1['image_name'] = sample_submission1['image_name'].apply(lambda x: '{}.jpg'.format(x))
sample_submission1.head()

In [ ]:
#loading in the weights of the trained model so we can make predictions with it
model2.load_weights('best_model.hdf5')
#we divide the sample submission file into two splits, first test1_df which contains the first 40669 images 
test1_df = sample_submission1.iloc[:40669]['image_name'].reset_index().drop('index', axis =1)
test1_df.head()

In [ ]:
#initializing imagedatagenerator for the test images and also rescaling
test_image_gen = ImageDataGenerator(rescale = 1/255)


#creating a generator for the images found in the first test image files
test_generator1 = test_image_gen.flow_from_dataframe(dataframe=test1_df, 
                                                directory="../input/planets-dataset/planet/planet/test-jpg", 
                                                x_col="image_name", y_col=None, batch_size=16, 
                                                shuffle=False, class_mode=None, target_size=(128,128))

step_test_size1 = int(np.ceil(test_generator1.samples/test_generator1.batch_size))

In [ ]:
#first, we reset the test generator to avoid shuffling of index as we want it to be orderly
test_generator1.reset()
pred1 = model2.predict(test_generator1, steps = step_test_size1, verbose = 1)

In [ ]:
#this is to get the filenames in the generator using the attribute .filenames
file_names1 = test_generator1.filenames

#convert the predicted values to a dataframe and join two labels together if the probability of occurrance 
#of the label is greater than 0.5 
pred_tags1 = pd.DataFrame(pred1)
pred_tags1 = pred_tags1.apply(lambda x: ' '.join(np.array(labels)[x>0.5]), axis = 1)

#then the result should look like this 
result1 = pd.DataFrame({'image_name': file_names1, 'tags': pred_tags1})
result1.head()

In [ ]:
#second batch of the test dataset
test2_df = sample_submission1.iloc[40669:]['image_name'].reset_index().drop('index', axis =1)
test2_df.head()

In [ ]:
#creating a generator for the second batch of test image files
test_generator2 = test_image_gen.flow_from_dataframe(dataframe=test2_df, 
                                                directory="../input/planets-dataset/test-jpg-additional/test-jpg-additional", 
                                                x_col="image_name", y_col=None, batch_size=16, 
                                                shuffle=False, class_mode=None, target_size=(128,128))

step_test_size2 = int(np.ceil(test_generator2.samples/test_generator2.batch_size))

In [ ]:
#we reset the generator to avoid shuffling, then make prediction on the generator
test_generator2.reset()
pred2 = model2.predict(test_generator2, steps = step_test_size2, verbose = 1)

In [ ]:
#this is to get the filenames in the generator using the attribute .filenames
file_names2 = test_generator2.filenames

#convert the predicted values to a dataframe and join two labels together if the probability of occurrance 
#of the label is greater than 0.5
pred_tags2 = pd.DataFrame(pred2)
pred_tags2 = pred_tags2.apply(lambda x: ''.join(np.array(labels)[x>0.5]), axis = 1)

#then the result should look like this
result2 = pd.DataFrame({'image_name': file_names2, 'tags': pred_tags2})
result2.head()

In [ ]:
#for the final result of the predicted tags for the test images, we need to concat the first and second results in 
#that order to avoid shuffling the index
last_result = pd.concat([result1, result2])

last_result = last_result.reset_index().drop('index', axis =1)

print(last_result.shape)
#print the final result
last_result.head()

In [ ]:
#we need to remove the .jpg extension from the image_name of the last_result because from the sample submission the 
#extension was not there, we added it for easy manipulation of the data.
last_result['image_name'] = last_result['image_name'].apply(lambda x: x[:-4])
last_result.head()